In [33]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.svm import SVC as SVM
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid as NC
from sklearn.model_selection import train_test_split as data_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [34]:
classes = {
	"image1"   : 1,
	"image2"   : 2,
	"image3"     : 3,
	"image4" : 4
}

classifiers = {
	"NC"         : NC(),
	"LDA"        : LDA(),
	"QDA"        : QDA(),
	"SVM_linear" : SVM(kernel="linear"),
	"SVM_radial" : SVM(kernel="rbf")
}

In [35]:
def feature_extraction(data_file, dataset="./Dataset/"):
	dump = []
	
	print("Extracting Hu moments...")
	
	for c, idx in classes.items():
		class_folder = dataset + "{}/".format(c)
		
		for f in os.listdir(class_folder):
			fpath = class_folder + f
			sample = (f.replace(".jpeg", ""))
			
			img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
			img = cv2.bitwise_not(img)
			hu = cv2.HuMoments(cv2.moments(img))
			print(f"moments:{f}->", hu)
			
			for i in range(0, len(hu)):
				hu[i] = -1 * np.sign(hu[i]) * np.log10(np.abs(hu[i]))
				# dump.append(hu[i])
			
			hu = hu.reshape((1, 7)).tolist()[0] + [sample, idx]
			dump.append(hu)
		
		print(c, "ok!")

	cols = ["hu1", "hu2", "hu3", "hu4", "hu5", "hu6", "hu7", "sample", "class"]
	
	df = pd.DataFrame(dump, columns=cols)
	df.to_csv(data_file, index=None)
	
	print("Extraction done!")

In [36]:
data_file = "hu_moments.csv"

feature_extraction(data_file)

Extracting Hu moments...
moments:image1.jpeg-> [[ 1.67680488e-03]
 [ 2.93962815e-07]
 [ 1.50143928e-11]
 [ 8.64448263e-12]
 [-7.87032918e-23]
 [-2.24166603e-15]
 [-5.92008737e-23]]
moments:image4.png-> [[ 7.41473066e-04]
 [ 4.28370218e-08]
 [ 2.36351125e-16]
 [ 2.24273276e-16]
 [ 4.99644182e-32]
 [ 4.44668292e-20]
 [-1.30284981e-32]]
image1 ok!
moments:image2.png-> [[ 1.46662174e-03]
 [ 1.02351178e-06]
 [ 1.24359918e-12]
 [ 8.45520750e-13]
 [ 6.32178482e-25]
 [ 2.63412632e-16]
 [-5.93350993e-25]]
moments:image5.png-> [[ 1.26128260e-03]
 [ 7.72926823e-07]
 [ 6.55736775e-12]
 [ 4.29086160e-12]
 [ 1.66813439e-23]
 [ 1.04294167e-15]
 [-1.54845887e-23]]
image2 ok!
moments:image3.png-> [[1.74025082e-03]
 [7.47256769e-07]
 [2.34345656e-12]
 [6.38733975e-12]
 [2.41719497e-23]
 [5.24567046e-15]
 [5.13823249e-24]]
image3 ok!
moments:original_min.png-> [[ 1.67711355e-03]
 [ 2.93165187e-07]
 [ 1.40596540e-11]
 [ 8.76778033e-12]
 [-7.61847058e-23]
 [-2.28876109e-15]
 [-6.05994685e-23]]
image4 ok!
E

In [37]:
def classification(data_file, rounds=100, remove_disperse=[]):
	df = pd.read_csv(data_file)
	df = df.drop(["sample"], axis=1)
	
	if remove_disperse:
		df = df.drop(remove_disperse, axis=1)
	
	X = df.drop(["class"], axis=1)
	y = df["class"]
	
	ans = {key: {"score" : []} for key, value in classifiers.items()}
	
	print("Classifying...")
	
	for i in range(rounds):
		X_train, X_test, y_train, y_test = data_split(X, y, test_size=0.3)
		
		for name, classifier in classifiers.items():
			scaler = StandardScaler()
			scaler.fit(X_train)
			X_train = scaler.transform(X_train)
			X_test = scaler.transform(X_test)
			
			classifier.fit(X_train, y_train)
			score = classifier.score(X_test, y_test)
			
			ans[name]["score"].append(score)
		
	print("Classification done!")
	
	return ans

In [38]:
ans = classification(data_file)

Classifying...


ValueError: y has only 1 sample in class 1, covariance is ill defined.